In [222]:
pip install unidecode

   ---------------------------------------- 0.0/235.5 kB ? eta -:--:--
   ----- ---------------------------------- 30.7/235.5 kB ? eta -:--:--
   --------------- ------------------------ 92.2/235.5 kB 1.1 MB/s eta 0:00:01
   --------------- ------------------------ 92.2/235.5 kB 1.1 MB/s eta 0:00:01
   -------------------------------------- - 225.3/235.5 kB 1.1 MB/s eta 0:00:01
   -------------------------------------- - 225.3/235.5 kB 1.1 MB/s eta 0:00:01
   -------------------------------------- - 225.3/235.5 kB 1.1 MB/s eta 0:00:01
   -------------------------------------- - 225.3/235.5 kB 1.1 MB/s eta 0:00:01
   -------------------------------------- - 225.3/235.5 kB 1.1 MB/s eta 0:00:01
   -------------------------------------- - 225.3/235.5 kB 1.1 MB/s eta 0:00:01
   -------------------------------------- - 225.3/235.5 kB 1.1 MB/s eta 0:00:01
   -------------------------------------- - 225.3/235.5 kB 1.1 MB/s eta 0:00:01
   -------------------------------------- 235.5/235.5 kB 42


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\benjamin\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [305]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 30)
from matplotlib import pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

import unidecode

In [619]:
df_cust = pd.read_csv('Dataset/olist_customers_dataset.csv')                        # Customers
df_geo = pd.read_csv('Dataset/olist_geolocation_dataset.csv')                       # Geolocation
df_order_items = pd.read_csv('Dataset/olist_order_items_dataset.csv')               # Order Items
df_order_payments = pd.read_csv('Dataset/olist_order_payments_dataset.csv')         # Order Payments
df_order_reviews = pd.read_csv('Dataset/olist_order_reviews_dataset.csv')           # Order Reviews
df_order = pd.read_csv('Dataset/olist_orders_dataset.csv')                          # Order
df_products = pd.read_csv('Dataset/olist_products_dataset.csv')                     # Products
df_sellers = pd.read_csv('Dataset/olist_sellers_dataset.csv')                       # Sellers
df_product_category = pd.read_csv('Dataset/product_category_name_translation.csv')  # Product Category

In [180]:
df_cust.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   customer_id               99441 non-null  object
 1   customer_unique_id        99441 non-null  object
 2   customer_zip_code_prefix  99441 non-null  int64 
 3   customer_city             99441 non-null  object
 4   customer_state            99441 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.8+ MB


In [181]:
# display(df_cust.head())
# display(df_order.head())
# display(df_sellers.head())
# display(df_order_items.head())
# display(df_order_reviews.head())
# display(df_order_payments.head())
# display(df_products.head())
# display(df_geo.head())
# display(df_product_category.head())

## **1. Products Dataset Cleaning**

Tables used:
1. Products
2. Products Category

Output DataFrame Variable: `products`

In [423]:
# Null Check
print(f'Products:\n{df_products.isna().sum()}')
print(f'\nProducts Category:\n{df_product_category.isna().sum()}')

Products:
product_id                    0
product_category_name         0
product_name_lenght           0
product_description_lenght    0
product_photos_qty            0
product_weight_g              1
product_length_cm             1
product_height_cm             1
product_width_cm              1
dtype: int64

Products Category:
product_category_name            0
product_category_name_english    0
dtype: int64


In [426]:
# Random Check On Products Dataset

# filtered_df = df_products[pd.isna(df_products['product_category_name'])]
filtered_df = df_products[pd.isna(df_products['product_weight_g'])]
filtered_df

# Drop Null Values
df_products.dropna(subset = ['product_category_name'], inplace = True)

# Fill Null Values with 0
df_products.fillna(0, inplace = True)

display(df_products.head())
display(df_products.info())

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0


<class 'pandas.core.frame.DataFrame'>
Index: 32341 entries, 0 to 32950
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   product_id                  32341 non-null  object 
 1   product_category_name       32341 non-null  object 
 2   product_name_lenght         32341 non-null  float64
 3   product_description_lenght  32341 non-null  float64
 4   product_photos_qty          32341 non-null  float64
 5   product_weight_g            32341 non-null  float64
 6   product_length_cm           32341 non-null  float64
 7   product_height_cm           32341 non-null  float64
 8   product_width_cm            32341 non-null  float64
dtypes: float64(7), object(2)
memory usage: 2.5+ MB


None

In [435]:
products = pd.merge(df_products, df_product_category,
                    how = 'left', on = 'product_category_name')

# Check for products which don;t have translator yet
products[pd.isna(products['product_category_name_english'])][['product_category_name', 'product_category_name_english']]

,product_category_name,product_category_name_english
1581,pc_gamer,NaN
5703,portateis_cozinha_e_preparadores_de_alimentos,NaN
7176,portateis_cozinha_e_preparadores_de_alimentos,NaN
7326,pc_gamer,NaN
8648,portateis_cozinha_e_preparadores_de_alimentos,NaN
10823,portateis_cozinha_e_preparadores_de_alimentos,NaN
13996,portateis_cozinha_e_preparadores_de_alimentos,NaN
15875,portateis_cozinha_e_preparadores_de_alimentos,NaN
16609,pc_gamer,NaN
17460,portateis_cozinha_e_preparadores_de_alimentos,NaN


In [437]:
# There are two product categories in portuguese which don't have the translate yet:
# portateis_cozinha_e_preparadores_de_alimentos	- portable_kitchen_and_food_preparation_tools
# pc_gamer - gamer_pc

add_categories = pd.DataFrame({
    'product_category_name' : ['portateis_cozinha_e_preparadores_de_alimentos', 'pc_gamer'],
    'product_category_name_english' : ['portable_kitchen_and_food_preparation_tools', 'gamer_pc']
})

df_product_category = pd.concat([df_product_category, add_categories],
                                ignore_index = True)

df_product_category

,product_category_name,product_category_name_english
0,beleza_saude,health_beauty
1,informatica_acessorios,computers_accessories
2,automotivo,auto
3,cama_mesa_banho,bed_bath_table
4,moveis_decoracao,furniture_decor
...,...,...
68,fraldas_higiene,diapers_and_hygiene
69,fashion_roupa_infanto_juvenil,fashion_childrens_clothes
70,seguros_e_servicos,security_and_services
71,portateis_cozinha_e_preparadores_de_alimentos,portable_kitchen_and_food_preparation_tools


In [438]:
# Re - Merge
products = pd.merge(df_products, df_product_category,
                    how = 'left', on = 'product_category_name')

In [439]:
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32341 entries, 0 to 32340
Data columns (total 10 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   product_id                     32341 non-null  object 
 1   product_category_name          32341 non-null  object 
 2   product_name_lenght            32341 non-null  float64
 3   product_description_lenght     32341 non-null  float64
 4   product_photos_qty             32341 non-null  float64
 5   product_weight_g               32341 non-null  float64
 6   product_length_cm              32341 non-null  float64
 7   product_height_cm              32341 non-null  float64
 8   product_width_cm               32341 non-null  float64
 9   product_category_name_english  32341 non-null  object 
dtypes: float64(7), object(3)
memory usage: 2.5+ MB


In [443]:
# Drop Product Category Name in Portuguese
products.drop(columns = 'product_category_name', inplace = True)

# Renaming Product Category Name in English
products.rename(columns = {'product_category_name_english' : 'product_cat_name'}, inplace = True)

# Reordering Columns
new_col_order = ['product_id', 'product_cat_name', 'product_name_lenght', 
                  'product_description_lenght', 'product_photos_qty', 'product_weight_g', 
                  'product_length_cm', 'product_height_cm', 'product_width_cm']

products = products[new_col_order]
products.head()

,product_id,product_cat_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumery,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,art,44.0,276.0,1.0,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,sports_leisure,46.0,250.0,1.0,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,baby,27.0,261.0,1.0,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,housewares,37.0,402.0,4.0,625.0,20.0,17.0,13.0


## **2. Geolocation Dataset Cleaning**

Tables used:
1. Geolocation

Output DataFrame Variable : `geolocation`

In [228]:
# Normalizing Cities Name Formats
df_geo.geolocation_city = df_geo.geolocation_city.apply(lambda x: unidecode.unidecode(x))

In [229]:
df_geo_zip_codes = df_geo['geolocation_zip_code_prefix'].unique()

df_zip_code_desc = pd.DataFrame(columns = ['Zip Code', 'Min Lat', 'Max Lat', 'Lat Diff', 'Median Lat',
                                           'Min Lng', 'Max Lng', 'Lng Diff', 'Median Lng',
                                           'Count', 'State', 'City'])

geo_list = []

def compute_geo_stats(df):
    min_lat = df['geolocation_lat'].min()
    max_lat = df['geolocation_lat'].max()
    median_lat = df['geolocation_lat'].median()
    min_lng = df['geolocation_lng'].min()
    max_lng = df['geolocation_lng'].max()
    median_lng = df['geolocation_lng'].median()
    
    return min_lat, max_lat, median_lat, min_lng, max_lng, median_lng

for zip_code in df_geo_zip_codes:
    df_temp = df_geo[df_geo['geolocation_zip_code_prefix'] == zip_code]
    states = df_temp.geolocation_state.unique()
    
    for state in states:
        df_temp_states = df_temp[df_temp['geolocation_state'] == state]
        cities = df_temp_states.geolocation_city.unique()
        
        for city in cities:
            df_temp_states_city = df_temp_states[df_temp_states['geolocation_city'] == city]
            min_lat, max_lat, median_lat, min_lng, max_lng, median_lng = compute_geo_stats(df_temp_states_city)
            geo_list.append({
                'Zip Code': zip_code,
                'Min Lat': min_lat,
                'Max Lat': max_lat,
                'Lat Diff': max_lat - min_lat,
                'Median Lat': median_lat,
                'Min Lng': min_lng,
                'Max Lng': max_lng,
                'Lng Diff': max_lng - min_lng,
                'Median Lng': median_lng,
                'Count': len(df_temp_states_city),
                'State': df_temp_states_city.geolocation_state.unique()[0],
                'City': df_temp_states_city.geolocation_city.unique()[0]
            })
    
df_zip_code_desc = pd.concat([df_zip_code_desc, pd.DataFrame(geo_list)], 
                             ignore_index = True)

display(df_zip_code_desc)

,Zip Code,Min Lat,Max Lat,Lat Diff,Median Lat,Min Lng,Max Lng,Lng Diff,Median Lng,Count,State,City
0,1037,-23.546723,-23.543137,0.003586,-23.545410,-46.640336,-46.637609,0.002728,-46.638819,26,SP,sao paulo
1,1046,-23.547191,-23.545048,0.002143,-23.546081,-46.645771,-46.642719,0.003052,-46.643810,141,SP,sao paulo
2,1041,-23.544638,-23.543208,0.001431,-23.543713,-46.640906,-46.639152,0.001754,-46.640263,25,SP,sao paulo
3,1035,-23.543947,-23.540205,0.003743,-23.540984,-46.644147,-46.637397,0.006751,-46.642569,39,SP,sao paulo
4,1012,-23.549008,-23.546640,0.002369,-23.547742,-46.635361,-46.634526,0.000834,-46.634881,17,SP,sao paulo
...,...,...,...,...,...,...,...,...,...,...,...,...
19613,99955,-28.144740,-28.099471,0.045269,-28.107588,-52.147869,-52.132700,0.015168,-52.145536,12,RS,vila langaro
19614,99970,-28.348616,-28.341329,0.007287,-28.343273,-51.879243,-51.870397,0.008846,-51.873734,21,RS,ciriaco
19615,99910,-27.863500,-27.847065,0.016435,-27.861309,-52.085128,-52.028621,0.056507,-52.084405,6,RS,floriano peixoto
19616,99920,-27.860664,-27.848551,0.012113,-27.854983,-52.304946,-52.298330,0.006616,-52.300271,8,RS,erebango


In [233]:
# df_zip_code_desc.City.unique()

In [231]:
print(df_zip_code_desc['Lat Diff'].min())
print(df_zip_code_desc['Lat Diff'].max())
print(df_zip_code_desc['Lng Diff'].min())
print(df_zip_code_desc['Lng Diff'].max())

0.0
77.02570802658605
0.0
72.55231470851325


In [239]:
geolocation = df_zip_code_desc[['Zip Code', 'State', 'Median Lat', 'Median Lng', 'City']]
geolocation.rename(columns = {'Median Lat' : 'Latitude',
                              'Median Lng': 'Longitude'},
                   inplace = True)
geolocation

,Zip Code,State,Latitude,Longitude,City
0,1037,SP,-23.545410,-46.638819,sao paulo
1,1046,SP,-23.546081,-46.643810,sao paulo
2,1041,SP,-23.543713,-46.640263,sao paulo
3,1035,SP,-23.540984,-46.642569,sao paulo
4,1012,SP,-23.547742,-46.634881,sao paulo
...,...,...,...,...,...
19613,99955,RS,-28.107588,-52.145536,vila langaro
19614,99970,RS,-28.343273,-51.873734,ciriaco
19615,99910,RS,-27.861309,-52.084405,floriano peixoto
19616,99920,RS,-27.854983,-52.300271,erebango


## **3. Order Dataset Cleaning**

Tables used:
1. Products (Already Cleaned)
1. Order
2. Order Items
3. Order Payments
4. Order Reviews

Output DataFrame Variable : `order_grouped`

In [445]:
display(products.head())
display(df_order.head())
display(df_order_items.head())
display(df_order_payments.head())
display(df_order_reviews.head())

,product_id,product_cat_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumery,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,art,44.0,276.0,1.0,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,sports_leisure,46.0,250.0,1.0,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,baby,27.0,261.0,1.0,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,housewares,37.0,402.0,4.0,625.0,20.0,17.0,13.0


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14


,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39
2,25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.71
3,ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.78
4,42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,128.45


,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10 00:00:00,2018-03-11 03:05:13
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN,2018-02-17 00:00:00,2018-02-18 14:36:24
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53


In [512]:
# Check Unique Order ID

print(f'Order Unique order_id count : {len(df_order.order_id.unique())}')
print(f'Order Items Unique order_id count : {len(df_order_items.order_id.unique())}')
print(f'Order Payments Unique order_id count : {len(df_order_payments.order_id.unique())}')
print(f'Order Reviews Unique order_id count : {len(df_order_reviews.order_id.unique())}')

Order Unique order_id count : 99441
Order Items Unique order_id count : 98666
Order Payments Unique order_id count : 99440
Order Reviews Unique order_id count : 98673


In [517]:
# Checking Order IDs pairs in excel
order_ids = [df_order['order_id'].unique(), 
             df_order_items['order_id'].unique(), 
             df_order_payments['order_id'].unique(), 
             df_order_reviews['order_id'].unique()]

df_order_ids = pd.DataFrame(order_ids).T
df_order_ids.columns = ['order', 'order_items', 'order_payments', 'order_reviews']

display(df_order_ids)

# To Excel
df_order_ids.to_excel('order_ids.xlsx')

,order,order_items,order_payments,order_reviews
0,e481f51cbdc54678b7cc49136f2d6af7,00010242fe8c5a6d1ba2dd792cb16214,b81ef226f3fe1789b1e8b2acac839d17,73fc7af87114b39712e6da79b0a377eb
1,53cdb2fc8bc7dce0b6741e2150273451,00018f77f2f0320c557190d7a144bdd3,a9810da82917af2d9aefd1278f1dcfa0,a548910a1c6147796b98fdf73dbeba33
2,47770eb9100c2d0c44946d9cf07ec65d,000229ec398224ef6ca0657da4fc703e,25e8ea4e93396b6fa0d3dd708e76c1bd,f9e4b658b201a9f2ecdecbb34bed034b
3,949d5b44dbf5de918fe9c16f97b45f8a,00024acbcdf0a6daa1e931b038114c75,ba78997921bbcdc1373bb41e913ab953,658677c97b385a9be170737859d3511b
4,ad21c59c0840e6cb83a9ceb5573f8159,00042b26cf59d7ce69dfabb4e55b4fd9,42fdf880ba16b47b59251dd489d4441a,8e6bfb81e283fa7e4f11123a3fb894f1
...,...,...,...,...
99436,9c5dedf39a927c1b2549525ed64a053c,None,7b905861d7c825891d6347454ea7863f,None
99437,63943bddc261676b46f01ca7ac2f7bd8,None,32609bbb3dd69b3c066a6860554a77bf,None
99438,83c1379a015df1e13d02aae0204711ab,None,b8b61059626efa996a60be9bb9320e10,None
99439,11c177c8e97725db2631073c19f07b62,None,28bbae6599b09d39ca406b747b6632b1,None


Checking Result:
> All order_id in Order Items data is in Order data (Order Items C Order)
> All order_id in Order Payments data is in Order data (Order Items C Order)
> All order_id in Order Reviews data is in Order data (Order Items C Order)

If later there are null value because order_id, we can simply drop the null values with the corresponding subset columns

In [611]:
df_order.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   order_id                       99441 non-null  object
 1   customer_id                    99441 non-null  object
 2   order_status                   99441 non-null  object
 3   order_purchase_timestamp       99441 non-null  object
 4   order_approved_at              99281 non-null  object
 5   order_delivered_carrier_date   97658 non-null  object
 6   order_delivered_customer_date  96476 non-null  object
 7   order_estimated_delivery_date  99441 non-null  object
dtypes: object(8)
memory usage: 6.1+ MB


In [620]:
print(f'order_status unique on NaN Approved Date: \n{df_order[pd.isna(df_order.order_approved_at) == True].order_status.unique()}')
print('\n')
print(f'order_status unique on non - NaN Approved Date: \n{df_order[pd.isna(df_order.order_approved_at) == False].order_status.unique()}')

order_status unique on NaN Approved Date: 
['canceled' 'delivered' 'created']


order_status unique on non - NaN Approved Date: 
['delivered' 'invoiced' 'shipped' 'processing' 'unavailable' 'canceled'
 'approved']


In [621]:
# Dropna on with subset order_approved_at
df_order.dropna(subset = 'order_approved_at', inplace = True)
df_order.dropna(inplace = True)
df_order.info()

<class 'pandas.core.frame.DataFrame'>
Index: 96461 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   order_id                       96461 non-null  object
 1   customer_id                    96461 non-null  object
 2   order_status                   96461 non-null  object
 3   order_purchase_timestamp       96461 non-null  object
 4   order_approved_at              96461 non-null  object
 5   order_delivered_carrier_date   96461 non-null  object
 6   order_delivered_customer_date  96461 non-null  object
 7   order_estimated_delivery_date  96461 non-null  object
dtypes: object(8)
memory usage: 6.6+ MB


In [648]:
order_items = pd.merge(df_order, df_order_items,
                       how = 'left', on ='order_id')
# No NaN values
display(order_items.head())
display(order_items.info())

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,1,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017-10-06 11:07:15,29.99,8.72
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,1,595fac2a385ac33a80bd5114aec74eb8,289cdb325fb7e7f891c38608bf9e0962,2018-07-30 03:24:27,118.70,22.76
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00,1,aa4383b373c6aca5d8797843e5594415,4869f7a5dfa277a7dca6462dcf3b52b2,2018-08-13 08:55:23,159.90,19.22
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00,1,d0b61bfb1de832b15ba9d266ca96e5b0,66922902710d126a0e7d26b0e3805106,2017-11-23 19:45:59,45.00,27.20
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00,1,65266b2da20d04dbe00c5c2d3bb7859e,2c9e548be18521d1c43cde1c582c6de8,2018-02-19 20:31:37,19.90,8.72


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110180 entries, 0 to 110179
Data columns (total 14 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   order_id                       110180 non-null  object 
 1   customer_id                    110180 non-null  object 
 2   order_status                   110180 non-null  object 
 3   order_purchase_timestamp       110180 non-null  object 
 4   order_approved_at              110180 non-null  object 
 5   order_delivered_carrier_date   110180 non-null  object 
 6   order_delivered_customer_date  110180 non-null  object 
 7   order_estimated_delivery_date  110180 non-null  object 
 8   order_item_id                  110180 non-null  int64  
 9   product_id                     110180 non-null  object 
 10  seller_id                      110180 non-null  object 
 11  shipping_limit_date            110180 non-null  object 
 12  price                         

None

In [668]:
# No New Rows, Clear
df_order_products = pd.merge(order_items, products,
                             how = 'left', on = 'product_id')

# Cleaning
df_order_products.drop(columns = ['product_id'], 
                       inplace = True)

display(df_order_products.head())
display(df_order_products.info())

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_item_id,seller_id,shipping_limit_date,price,freight_value,product_cat_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,1,3504c0cb71d7fa48d967e0e4c94d59d9,2017-10-06 11:07:15,29.99,8.72,housewares,40.0,268.0,4.0,500.0,19.0,8.0,13.0
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,1,289cdb325fb7e7f891c38608bf9e0962,2018-07-30 03:24:27,118.70,22.76,perfumery,29.0,178.0,1.0,400.0,19.0,13.0,19.0
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00,1,4869f7a5dfa277a7dca6462dcf3b52b2,2018-08-13 08:55:23,159.90,19.22,auto,46.0,232.0,1.0,420.0,24.0,19.0,21.0
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00,1,66922902710d126a0e7d26b0e3805106,2017-11-23 19:45:59,45.00,27.20,pet_shop,59.0,468.0,3.0,450.0,30.0,10.0,20.0
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00,1,2c9e548be18521d1c43cde1c582c6de8,2018-02-19 20:31:37,19.90,8.72,stationery,38.0,316.0,4.0,250.0,51.0,15.0,15.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110180 entries, 0 to 110179
Data columns (total 21 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   order_id                       110180 non-null  object 
 1   customer_id                    110180 non-null  object 
 2   order_status                   110180 non-null  object 
 3   order_purchase_timestamp       110180 non-null  object 
 4   order_approved_at              110180 non-null  object 
 5   order_delivered_carrier_date   110180 non-null  object 
 6   order_delivered_customer_date  110180 non-null  object 
 7   order_estimated_delivery_date  110180 non-null  object 
 8   order_item_id                  110180 non-null  int64  
 9   seller_id                      110180 non-null  object 
 10  shipping_limit_date            110180 non-null  object 
 11  price                          110180 non-null  float64
 12  freight_value                 

None

In [669]:
unique_product_ids = set(products['product_id'].unique())
unique_order_items = order_items['product_id'].unique()

print(len(unique_product_ids))
print(len(unique_order_items))

# x = np.isin(unique_order_items, unique_product_ids).sum() #34.4 secs

x = sum(1 for item in unique_order_items if item in unique_product_ids)

print(x)

32341
32210
31626


In [671]:
# Replacing NaN Values
df_order_products.product_cat_name.fillna('not_recorded', inplace = True)
df_order_products.iloc[:, 14:] = df_order_products.iloc[:, 14:].fillna(0)

# Parse Columns to Integer
cols_to_int = df_order_products.drop(['price', 'freight_value'],
                                     axis = 1).select_dtypes(include = float).columns

for cols in cols_to_int:
    df_order_products[cols] = df_order_products[cols].astype(int)

In [673]:
df_order_products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110180 entries, 0 to 110179
Data columns (total 21 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   order_id                       110180 non-null  object 
 1   customer_id                    110180 non-null  object 
 2   order_status                   110180 non-null  object 
 3   order_purchase_timestamp       110180 non-null  object 
 4   order_approved_at              110180 non-null  object 
 5   order_delivered_carrier_date   110180 non-null  object 
 6   order_delivered_customer_date  110180 non-null  object 
 7   order_estimated_delivery_date  110180 non-null  object 
 8   order_item_id                  110180 non-null  int64  
 9   seller_id                      110180 non-null  object 
 10  shipping_limit_date            110180 non-null  object 
 11  price                          110180 non-null  float64
 12  freight_value                 

Check why there are NaN values after merge, while after merge, no new rows added

In [666]:
print(f'Order Items data product_id unique count : {len(df_order_items.product_id.unique())}')
print(f'Products data product_id unique count : {len(products.product_id.unique())}')

Order Items data product_id unique count : 32951
Products data product_id unique count : 32341


In [667]:
nan_order_id = df_order_products[pd.isna(df_order_products['product_cat_name'])].order_id.unique()
print(f'order_id not in Products Dataset count: {len(nan_order_id)}')

order_id not in Products Dataset count: 1391


In [674]:
df_product_payments = pd.merge(df_order_products, df_order_payments,
                               how = 'left', on = 'order_id')

df_product_payments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115021 entries, 0 to 115020
Data columns (total 25 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   order_id                       115021 non-null  object 
 1   customer_id                    115021 non-null  object 
 2   order_status                   115021 non-null  object 
 3   order_purchase_timestamp       115021 non-null  object 
 4   order_approved_at              115021 non-null  object 
 5   order_delivered_carrier_date   115021 non-null  object 
 6   order_delivered_customer_date  115021 non-null  object 
 7   order_estimated_delivery_date  115021 non-null  object 
 8   order_item_id                  115021 non-null  int64  
 9   seller_id                      115021 non-null  object 
 10  shipping_limit_date            115021 non-null  object 
 11  price                          115021 non-null  float64
 12  freight_value                 

In [676]:
display(df_product_payments[pd.isna(df_product_payments.payment_value)])

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_item_id,seller_id,shipping_limit_date,price,freight_value,product_cat_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,payment_sequential,payment_type,payment_installments,payment_value
35611,bfbd0f9bdef84302105ad712db648a6c,86dc2ffce2dfff336de2f386a786e574,delivered,2016-09-15 12:16:38,2016-09-15 12:16:38,2016-11-07 17:11:53,2016-11-09 07:47:38,2016-10-04 00:00:00,1,ecccfa2bb93b34a3bf033cc5d1dcdc69,2016-09-19 23:11:33,44.99,2.83,health_beauty,34,1036,1,1000,16,16,16,NaN,NaN,NaN,NaN
35612,bfbd0f9bdef84302105ad712db648a6c,86dc2ffce2dfff336de2f386a786e574,delivered,2016-09-15 12:16:38,2016-09-15 12:16:38,2016-11-07 17:11:53,2016-11-09 07:47:38,2016-10-04 00:00:00,2,ecccfa2bb93b34a3bf033cc5d1dcdc69,2016-09-19 23:11:33,44.99,2.83,health_beauty,34,1036,1,1000,16,16,16,NaN,NaN,NaN,NaN
35613,bfbd0f9bdef84302105ad712db648a6c,86dc2ffce2dfff336de2f386a786e574,delivered,2016-09-15 12:16:38,2016-09-15 12:16:38,2016-11-07 17:11:53,2016-11-09 07:47:38,2016-10-04 00:00:00,3,ecccfa2bb93b34a3bf033cc5d1dcdc69,2016-09-19 23:11:33,44.99,2.83,health_beauty,34,1036,1,1000,16,16,16,NaN,NaN,NaN,NaN


In [678]:
# Order Id with NaN value : bfbd0f9bdef84302105ad712db648a6c

display(df_product_payments[df_product_payments.order_id == 'bfbd0f9bdef84302105ad712db648a6c'])

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_item_id,seller_id,shipping_limit_date,price,freight_value,product_cat_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,payment_sequential,payment_type,payment_installments,payment_value
35611,bfbd0f9bdef84302105ad712db648a6c,86dc2ffce2dfff336de2f386a786e574,delivered,2016-09-15 12:16:38,2016-09-15 12:16:38,2016-11-07 17:11:53,2016-11-09 07:47:38,2016-10-04 00:00:00,1,ecccfa2bb93b34a3bf033cc5d1dcdc69,2016-09-19 23:11:33,44.99,2.83,health_beauty,34,1036,1,1000,16,16,16,NaN,NaN,NaN,NaN
35612,bfbd0f9bdef84302105ad712db648a6c,86dc2ffce2dfff336de2f386a786e574,delivered,2016-09-15 12:16:38,2016-09-15 12:16:38,2016-11-07 17:11:53,2016-11-09 07:47:38,2016-10-04 00:00:00,2,ecccfa2bb93b34a3bf033cc5d1dcdc69,2016-09-19 23:11:33,44.99,2.83,health_beauty,34,1036,1,1000,16,16,16,NaN,NaN,NaN,NaN
35613,bfbd0f9bdef84302105ad712db648a6c,86dc2ffce2dfff336de2f386a786e574,delivered,2016-09-15 12:16:38,2016-09-15 12:16:38,2016-11-07 17:11:53,2016-11-09 07:47:38,2016-10-04 00:00:00,3,ecccfa2bb93b34a3bf033cc5d1dcdc69,2016-09-19 23:11:33,44.99,2.83,health_beauty,34,1036,1,1000,16,16,16,NaN,NaN,NaN,NaN


Order Id with Null payments : `bfbd0f9bdef84302105ad712db648a6c`, this order id may not have paid off yet

In [679]:
df_product_payments.dropna(inplace = True)
df_product_payments.info()

<class 'pandas.core.frame.DataFrame'>
Index: 115018 entries, 0 to 115020
Data columns (total 25 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   order_id                       115018 non-null  object 
 1   customer_id                    115018 non-null  object 
 2   order_status                   115018 non-null  object 
 3   order_purchase_timestamp       115018 non-null  object 
 4   order_approved_at              115018 non-null  object 
 5   order_delivered_carrier_date   115018 non-null  object 
 6   order_delivered_customer_date  115018 non-null  object 
 7   order_estimated_delivery_date  115018 non-null  object 
 8   order_item_id                  115018 non-null  int64  
 9   seller_id                      115018 non-null  object 
 10  shipping_limit_date            115018 non-null  object 
 11  price                          115018 non-null  float64
 12  freight_value                  1150

In [682]:
df_order_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99224 entries, 0 to 99223
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   review_id                99224 non-null  object
 1   order_id                 99224 non-null  object
 2   review_score             99224 non-null  int64 
 3   review_comment_title     11568 non-null  object
 4   review_comment_message   40977 non-null  object
 5   review_creation_date     99224 non-null  object
 6   review_answer_timestamp  99224 non-null  object
dtypes: int64(1), object(6)
memory usage: 5.3+ MB


In [685]:
df_order_reviews.review_score.unique()

array([4, 5, 1, 3, 2], dtype=int64)

In [684]:
order = pd.merge(df_product_payments, df_order_reviews,
                 how = 'left', on = 'order_id')

order.drop(columns = ['review_id', 'review_comment_title', 'review_comment_message'], 
           inplace = True)

order.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115703 entries, 0 to 115702
Data columns (total 28 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   order_id                       115703 non-null  object 
 1   customer_id                    115703 non-null  object 
 2   order_status                   115703 non-null  object 
 3   order_purchase_timestamp       115703 non-null  object 
 4   order_approved_at              115703 non-null  object 
 5   order_delivered_carrier_date   115703 non-null  object 
 6   order_delivered_customer_date  115703 non-null  object 
 7   order_estimated_delivery_date  115703 non-null  object 
 8   order_item_id                  115703 non-null  int64  
 9   seller_id                      115703 non-null  object 
 10  shipping_limit_date            115703 non-null  object 
 11  price                          115703 non-null  float64
 12  freight_value                 

In [690]:
# Parsing columns data types
# Convert to int
order.review_score.fillna(0, inplace = True)
order[['review_creation_date', 'review_answer_timestamp']].fillna("", inplace = True)

cols_to_int = order.select_dtypes(include = 'float').columns.to_list()

for cols in ['price', 'freight_value']:
    while cols in cols_to_int:
        cols_to_int.remove(cols)
        
for cols in cols_to_int:
    order[cols] = order[cols].astype(int)

# Convert to datetime (remove the timestamp)
# for cols in ['shipping_limit_date', 'review_creation_date', 'review_answer_timestamp']:
#     order[cols] = pd.to_datetime(order[cols]).dt.date

# display(order.head())
# display(order.info())

In [691]:
columns_to_keep = order.columns.to_list()
columns_to_keep.remove('order_item_id')
columns_to_keep.remove('order_id')

# Group by order_id and aggregate
order_grouped = order.groupby('order_id').agg({
    'order_item_id': 'count',                    # Count number of items
    **{col: 'first' for col in columns_to_keep}  # Keep first value for other columns
}).reset_index()

order_grouped.rename(columns={'order_item_id': 'items_count'}, inplace=True)

display(order_grouped)

,order_id,items_count,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,seller_id,shipping_limit_date,price,freight_value,product_cat_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,payment_sequential,payment_type,payment_installments,payment_value,review_score,review_creation_date,review_answer_timestamp
0,00010242fe8c5a6d1ba2dd792cb16214,1,3ce436f183e68e07877b285a838db11a,delivered,2017-09-13 08:59:02,2017-09-13 09:45:35,2017-09-19 18:34:16,2017-09-20 23:43:48,2017-09-29 00:00:00,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29,cool_stuff,58,598,4,650,28,9,14,1,credit_card,2,72,5,2017-09-21 00:00:00,2017-09-22 10:57:03
1,00018f77f2f0320c557190d7a144bdd3,1,f6dd3ec061db4e3987629fe6b26e5cce,delivered,2017-04-26 10:53:06,2017-04-26 11:05:13,2017-05-04 14:35:00,2017-05-12 16:04:24,2017-05-15 00:00:00,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93,pet_shop,56,239,2,30000,50,30,40,1,credit_card,3,259,4,2017-05-13 00:00:00,2017-05-15 11:34:13
2,000229ec398224ef6ca0657da4fc703e,1,6489ae5e4333f3693df5ad4372dab6d3,delivered,2018-01-14 14:33:31,2018-01-14 14:48:30,2018-01-16 12:36:48,2018-01-22 13:19:16,2018-02-05 00:00:00,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87,furniture_decor,59,695,2,3050,33,13,33,1,credit_card,5,216,5,2018-01-23 00:00:00,2018-01-23 16:06:31
3,00024acbcdf0a6daa1e931b038114c75,1,d4eb9395c8c0431ee92fce09860c5a06,delivered,2018-08-08 10:00:35,2018-08-08 10:10:18,2018-08-10 13:28:00,2018-08-14 13:32:39,2018-08-20 00:00:00,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79,perfumery,42,480,1,200,16,10,15,1,credit_card,2,25,4,2018-08-15 00:00:00,2018-08-15 16:39:01
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,58dbd0b2d70206bf40e62cd34e84d795,delivered,2017-02-04 13:57:51,2017-02-04 14:10:13,2017-02-16 09:46:09,2017-03-01 16:42:31,2017-03-17 00:00:00,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14,garden_tools,59,409,1,3750,35,40,30,1,credit_card,3,218,5,2017-03-02 00:00:00,2017-03-03 10:54:59
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96455,fffc94f6ce00a00581880bf54a75a037,1,b51593916b4b8e0d6f66f2ae24f2673d,delivered,2018-04-23 13:57:06,2018-04-25 04:11:01,2018-04-25 12:09:00,2018-05-10 22:56:40,2018-05-18 00:00:00,b8bc237ba3788b23da09c0f1f3a3288c,2018-05-02 04:11:01,299.99,43.41,housewares,43,1002,3,10150,89,15,40,1,boleto,1,343,5,2018-05-11 00:00:00,2018-05-14 12:53:47
96456,fffcd46ef2263f404302a634eb57f7eb,1,84c5d4fbaf120aae381fad077416eaa0,delivered,2018-07-14 10:26:46,2018-07-17 04:31:48,2018-07-17 08:05:00,2018-07-23 20:31:55,2018-08-01 00:00:00,f3c38ab652836d21de61fb8314b69182,2018-07-20 04:31:48,350.00,36.53,computers_accessories,31,232,1,8950,45,26,38,1,boleto,1,386,5,2018-07-24 00:00:00,2018-07-25 09:25:29
96457,fffce4705a9662cd70adb13d4a31832d,1,29309aa813182aaddc9b259e31b870e6,delivered,2017-10-23 17:07:56,2017-10-24 17:14:25,2017-10-26 15:13:14,2017-10-28 12:22:22,2017-11-10 00:00:00,c3cfdc648177fdbbbb35635a37472c53,2017-10-30 17:14:25,99.90,16.95,sports_leisure,43,869,1,967,21,24,19,1,credit_card,3,116,5,2017-10-29 00:00:00,2017-10-29 21:33:52
96458,fffe18544ffabc95dfada21779c9644f,1,b5e6afd5a41800fdf401e0272ca74655,delivered,2017-08-14 23:02:59,2017-08-15 00:04:32,2017-08-15 19:02:53,2017-08-16 21:59:40,2017-08-25 00:00:00,2b3e4a2a3ea8e01938cabda2a3e5cc79,2017-08-21 00:04:32,55.99,8.72,computers_accessories,56,1306,1,100,20,20,20,1,credit_card,3,64,5,2017-08-17 00:00:00,2017-08-18 12:24:05


#### order_id filtering

In [ ]:
df_order_products[df_order_products.order_id == '848b0561d4bc26f57b288a5004f38683']
# df_order_products[df_order_products.order_id == '848b0561d4bc26f57b288a5004f38683']

In [ ]:
df_order_products[(df_order_products.order_item_id > 10)].sample(10)

## **4. Customer & Seller**

Tables used:
1. Order (Already Cleaned)
2. Geolocation (Already Cleaned)
3. Customer
4. Sellers

Output DataFrame Variable : `NaN`

In [567]:
display(df_cust.info())
display(df_sellers.info())
display(order.info())
display(geolocation.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   customer_id               99441 non-null  object
 1   customer_unique_id        99441 non-null  object
 2   customer_zip_code_prefix  99441 non-null  int64 
 3   customer_city             99441 non-null  object
 4   customer_state            99441 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.8+ MB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3095 entries, 0 to 3094
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   seller_id               3095 non-null   object
 1   seller_zip_code_prefix  3095 non-null   int64 
 2   seller_city             3095 non-null   object
 3   seller_state            3095 non-null   object
dtypes: int64(1), object(3)
memory usage: 96.8+ KB


None

<class 'pandas.core.frame.DataFrame'>
Index: 115634 entries, 0 to 116600
Data columns (total 21 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   product_cat_name            115634 non-null  object 
 1   order_item_id               115634 non-null  int64  
 2   price                       115634 non-null  float64
 3   freight_value               115634 non-null  float64
 4   order_id                    115634 non-null  object 
 5   seller_id                   115634 non-null  object 
 6   shipping_limit_date         115634 non-null  object 
 7   product_name_lenght         115634 non-null  int32  
 8   product_description_lenght  115634 non-null  int32  
 9   product_photos_qty          115634 non-null  int32  
 10  product_weight_g            115634 non-null  int32  
 11  product_length_cm           115634 non-null  int32  
 12  product_height_cm           115634 non-null  int32  
 13  product_width_cm   

None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19618 entries, 0 to 19617
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Zip Code   19618 non-null  object 
 1   State      19618 non-null  object 
 2   Latitude   19618 non-null  float64
 3   Longitude  19618 non-null  float64
 4   City       19618 non-null  object 
dtypes: float64(2), object(3)
memory usage: 766.5+ KB


None

In [568]:
display(df_cust.head())
display(df_sellers.head())
display(order.head())
display(geolocation.head())

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP
2,ce3ad9de960102d0677a81f5d0bb7b2d,20031,rio de janeiro,RJ
3,c0f3eea2e14555b6faeea3dd58c1b1c3,4195,sao paulo,SP
4,51a04a8a6bdcb23deccc82b0b80742cf,12914,braganca paulista,SP


,product_cat_name,order_item_id,price,freight_value,order_id,seller_id,shipping_limit_date,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,payment_sequential,payment_type,payment_installments,payment_value,review_score,review_creation_date,review_answer_timestamp
0,cool_stuff,1,58.90,13.29,00010242fe8c5a6d1ba2dd792cb16214,48436dade18ac8b2bce089ec2a041202,2017-09-19,58,598,4,650,28,9,14,1,credit_card,2,72,5,2017-09-21,2017-09-22
1,pet_shop,1,239.90,19.93,00018f77f2f0320c557190d7a144bdd3,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03,56,239,2,30000,50,30,40,1,credit_card,3,259,4,2017-05-13,2017-05-15
2,furniture_decor,1,199.00,17.87,000229ec398224ef6ca0657da4fc703e,5b51032eddd242adc84c38acab88f23d,2018-01-18,59,695,2,3050,33,13,33,1,credit_card,5,216,5,2018-01-23,2018-01-23
3,perfumery,1,12.99,12.79,00024acbcdf0a6daa1e931b038114c75,9d7a1d34a5052409006425275ba1c2b4,2018-08-15,42,480,1,200,16,10,15,1,credit_card,2,25,4,2018-08-15,2018-08-15
4,garden_tools,1,199.90,18.14,00042b26cf59d7ce69dfabb4e55b4fd9,df560393f3a51e74553ab94004ba5c87,2017-02-13,59,409,1,3750,35,40,30,1,credit_card,3,218,5,2017-03-02,2017-03-03


,Zip Code,State,Latitude,Longitude,City
0,1037,SP,-23.545410,-46.638819,sao paulo
1,1046,SP,-23.546081,-46.643810,sao paulo
2,1041,SP,-23.543713,-46.640263,sao paulo
3,1035,SP,-23.540984,-46.642569,sao paulo
4,1012,SP,-23.547742,-46.634881,sao paulo


In [573]:
# No New Rows Added, Clear
customer = pd.merge(df_cust, geolocation,
                    how = 'left',
                    left_on = ['customer_zip_code_prefix', 'customer_city', 'customer_state'],
                    right_on = ['Zip Code', 'City', 'State'])

# Drop Columns
customer.drop(columns = ['Zip Code', 'City', 'State'],
              inplace = True)

# Rename Columns
customer.rename(columns = {
    'Latitude' : 'customer_lat',
    'Longitude' : 'customer_lng',
}, inplace = True)

customer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 7 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   customer_id               99441 non-null  object 
 1   customer_unique_id        99441 non-null  object 
 2   customer_zip_code_prefix  99441 non-null  object 
 3   customer_city             99441 non-null  object 
 4   customer_state            99441 non-null  object 
 5   customer_lat              99139 non-null  float64
 6   customer_lng              99139 non-null  float64
dtypes: float64(2), object(5)
memory usage: 5.3+ MB


In [574]:
# No New Rows Added, Clear
seller = pd.merge(df_sellers, geolocation,
                  how = 'left',
                  left_on = ['seller_zip_code_prefix', 'seller_city', 'seller_state'],
                  right_on = ['Zip Code', 'City', 'State'])

# Drop Columns
seller.drop(columns = ['Zip Code', 'City', 'State'],
            inplace = True)

# Rename Columns
seller.rename(columns = {
    'Latitude' : 'seller_lat',
    'Longitude' : 'seller_lng',
}, inplace = True)

seller.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3095 entries, 0 to 3094
Data columns (total 6 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   seller_id               3095 non-null   object 
 1   seller_zip_code_prefix  3095 non-null   object 
 2   seller_city             3095 non-null   object 
 3   seller_state            3095 non-null   object 
 4   seller_lat              2960 non-null   float64
 5   seller_lng              2960 non-null   float64
dtypes: float64(2), object(4)
memory usage: 145.2+ KB


In [576]:
geolocation

,Zip Code,State,Latitude,Longitude,City
0,1037,SP,-23.545410,-46.638819,sao paulo
1,1046,SP,-23.546081,-46.643810,sao paulo
2,1041,SP,-23.543713,-46.640263,sao paulo
3,1035,SP,-23.540984,-46.642569,sao paulo
4,1012,SP,-23.547742,-46.634881,sao paulo
...,...,...,...,...,...
19613,99955,RS,-28.107588,-52.145536,vila langaro
19614,99970,RS,-28.343273,-51.873734,ciriaco
19615,99910,RS,-27.861309,-52.084405,floriano peixoto
19616,99920,RS,-27.854983,-52.300271,erebango


In [577]:
df_cust

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP
...,...,...,...,...,...
99436,17ddf5dd5d51696bb3d7c6291687be6f,1a29b476fee25c95fbafc67c5ac95cf8,3937,sao paulo,SP
99437,e7b71a9017aa05c9a7fd292d714858e8,d52a67c98be1cf6a5c84435bd38d095d,6764,taboao da serra,SP
99438,5e28dfe12db7fb50a4b2f691faecea5e,e9f50caf99f032f0bf3c55141f019d99,60115,fortaleza,CE
99439,56b18e2166679b8a959d72dd06da27f9,73c2643a0a458b49f58cea58833b192e,92120,canoas,RS


In [581]:
print(len(geolocation.City.unique()))
print(len(df_cust.customer_city.unique()))

5969
27


In [594]:
print(len(geolocation.State.unique()))
print(len(df_cust.customer_state.unique()))

i = 0
for cust_state, geo_state in zip(sorted(df_cust.customer_state.unique()), sorted(geolocation.State.unique())):
    if cust_state == geo_state:
        i += 1

print(i)

27
27
27


In [595]:
print(len(geolocation['Zip Code'].unique()))
print(len(df_cust.customer_zip_code_prefix.unique()))

19015
14994


In [598]:
df_cust

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP
...,...,...,...,...,...
99436,17ddf5dd5d51696bb3d7c6291687be6f,1a29b476fee25c95fbafc67c5ac95cf8,3937,sao paulo,SP
99437,e7b71a9017aa05c9a7fd292d714858e8,d52a67c98be1cf6a5c84435bd38d095d,6764,taboao da serra,SP
99438,5e28dfe12db7fb50a4b2f691faecea5e,e9f50caf99f032f0bf3c55141f019d99,60115,fortaleza,CE
99439,56b18e2166679b8a959d72dd06da27f9,73c2643a0a458b49f58cea58833b192e,92120,canoas,RS


In [600]:
order.head()

,product_cat_name,order_item_id,price,freight_value,order_id,seller_id,shipping_limit_date,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,payment_sequential,payment_type,payment_installments,payment_value,review_score,review_creation_date,review_answer_timestamp
0,cool_stuff,1,58.90,13.29,00010242fe8c5a6d1ba2dd792cb16214,48436dade18ac8b2bce089ec2a041202,2017-09-19,58,598,4,650,28,9,14,1,credit_card,2,72,5,2017-09-21,2017-09-22
1,pet_shop,1,239.90,19.93,00018f77f2f0320c557190d7a144bdd3,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03,56,239,2,30000,50,30,40,1,credit_card,3,259,4,2017-05-13,2017-05-15
2,furniture_decor,1,199.00,17.87,000229ec398224ef6ca0657da4fc703e,5b51032eddd242adc84c38acab88f23d,2018-01-18,59,695,2,3050,33,13,33,1,credit_card,5,216,5,2018-01-23,2018-01-23
3,perfumery,1,12.99,12.79,00024acbcdf0a6daa1e931b038114c75,9d7a1d34a5052409006425275ba1c2b4,2018-08-15,42,480,1,200,16,10,15,1,credit_card,2,25,4,2018-08-15,2018-08-15
4,garden_tools,1,199.90,18.14,00042b26cf59d7ce69dfabb4e55b4fd9,df560393f3a51e74553ab94004ba5c87,2017-02-13,59,409,1,3750,35,40,30,1,credit_card,3,218,5,2017-03-02,2017-03-03
